In [5]:
import ibapi
from ibapi import wrapper
from ibapi.client import EClient
from ibapi.utils import iswrapper
from ibapi.contract import Contract
import datetime

In [6]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.ticktype import TickTypeEnum
from ibapi.common import TagValueList
from ibapi.tag_value import TagValue

import pandas as pd

import threading
import time
import datetime
import random
import math

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.hist_data = [] #Initialize variable to store candle
        self.acct_summary = []
        self.order_status = {}
        self.pending_orders = []
        self.positions = []
        self.trading_hours = {}
        self.tick_prices = {}
        
    def tickPrice(self, reqId, tickType, price, attrib):
        from ibapi.ticktype import TickTypeEnum
        #print("Tick ID : ", TickTypeEnum.to_str(int(tickType)), ", Price : ", price)
        self.tick_prices[TickTypeEnum.to_str(int(tickType))] = price
    
    def tickString(self, reqId, tickType, value):
        super().tickString(reqId, tickType, value)
        #print("TickString. TickerId:", reqId, "Type:", tickType, "Value:", value)
    
    def historicalData(self, reqId, bar):
        #print("Historical Data Req. Request ID : ", reqId)
        self.hist_data.append([bar.date, bar.open, bar.high, bar.low, bar.close, bar.average, bar.volume, bar.barCount])            

    def accountSummary(self, reqId, account, tag, value, currency):
        super().accountSummary(reqId, account, tag, value, currency)
        #print("Account Summary Data Req. Request ID : ", reqId)
        self.acct_summary.append([account, tag, value, currency])
        #print("AccountSummary. ReqId:", reqId, "Account:", account,"Tag: ", tag, "Value:", value, "Currency:", currency)
             
    def position(self, account, contract, position, avgCost):
        super().position(account, contract, position, avgCost)
        #print("Position.", "Account:", account, "Symbol:", contract.symbol, "SecType:",
        #      contract.secType, "Currency:", contract.currency,
        #      "Position:", position, "Avg cost:", avgCost)
        self.positions.append([account,contract.symbol,contract.secType,contract.currency,position,avgCost])
        
    def pnlSingle(self, reqId, pos, dailyPnL, unrealizedPnL, realizedPnL, value):
        super().pnlSingle(reqId, pos, dailyPnL, unrealizedPnL, realizedPnL, value)
        print("Daily PnL Single. ReqId:", reqId, "Position:", pos,
              "DailyPnL:", dailyPnL, "UnrealizedPnL:", unrealizedPnL,
              "RealizedPnL:", realizedPnL, "Value:", value)
        
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        self.order_status[orderId] = status
        print('orderStatus - orderid: ', orderId, 'status: ', status, 'filled ', filled, 'remaining ', remaining, 'lastFillPrice ', lastFillPrice)

    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)
        
    def openOrder(self, orderId, contract, order, orderState):
        super().openOrder(orderId, contract, order, orderState)
        #print("OpenOrder. PermId: ", order.permId, "ClientId:", order.clientId, " OrderId:", orderId, 
        #      "Account:", order.account, "Symbol:", contract.symbol, "SecType:", contract.secType,
        #      "Exchange:", contract.exchange, "Action:", order.action, "OrderType:", order.orderType,
        #      "TotalQty:", order.totalQuantity, "CashQty:", order.cashQty, 
        #      "LmtPrice:", order.lmtPrice, "AuxPrice:", order.auxPrice, "Status:", orderState.status)
        self.pending_orders.append([order.permId, order.clientId, orderId,order.account,contract.symbol,
                                    contract.secType,contract.exchange, order.action, order.orderType,
                                    order.totalQuantity, order.cashQty,order.lmtPrice,
                                    order.auxPrice, orderState.status])
        #order.contract = contract
        #self.permId2ord[order.permId] = order
    def openOrderEnd(self):
        super().openOrderEnd()
        print("OpenOrderEnd")
        #print("Received %d openOrders", len(self.permId2ord))
        
    def contractDetails(self, reqId, contractDetails):
        super().contractDetails(reqId, contractDetails)
        reg_session = contractDetails.liquidHours.split(";")[0].split("-")
        dt = reg_session[0].strip().split(":")[0]
        reg_session = " - ".join([d.split(":")[1] for d in reg_session])
        
        dt = datetime.datetime.strptime(dt, "%Y%m%d").strftime("%B %d - %Y")
        
        trading_session = contractDetails.tradingHours.split(";")[0].split("-")
        trading_session = " - ".join([d.split(":")[1] for d in trading_session])
        
        self.trading_hours["Date"] = dt
        self.trading_hours["Regular Trading Session"] = reg_session
        self.trading_hours["Total Available Hours"] = trading_session
        
        #print("Regular Trading Session : ", contractDetails.liquidHours.split(";")[0])
        
        #print("Total Available Hours   : ", contractDetails.tradingHours.split(";")[0])       

In [14]:
class Trading:
    def createContract(self, symbol, sec_type="STK", exchange="SMART", currency="USD"):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = sec_type
        contract.exchange = exchange
        contract.currency = currency

        return contract

    def createOrder(self, buy_sell="BUY", order_type="MKT", qty=1, duration="DAY", lmt_prc=""):
        order = Order()
        order.action = buy_sell
        order.orderType = order_type
        order.totalQuantity = qty
        order.Tif = duration
        order.outsideRth = True
        if order_type=="LMT" and lmt_prc:
            order.lmtPrice = lmt_prc

        return order 

    def apply_adaptive_algo_params(self, base_order, priority="Normal"):
        baseOrder.algoStrategy = "Adaptive" 
        baseOrder.algoParams = []
        baseOrder.algoParams.append(TagValue("adaptivePriority", priority))

    def start_app_in_background(self, app):
        #Start the socket in a thread
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()

        time.sleep(1) #Sleep interval to allow time for connection to server

    def retrieve_ohlc_data(self, ticker = "AAPL", start_date="", end_date='', bar_size="1 hour", measure="TRADES", sleep_time=5):
        if not start_date:
            print("Start Date Not Provided. Setting Start Date to Default 5 Days back from End Date.")
         
        try:
            dt2 = datetime.datetime.now() if not end_date else datetime.datetime.strptime(end_date, "%Y%m%d")
            dt2 = dt2 + datetime.timedelta(days=1)
            dt1 = datetime.datetime.strptime(start_date, "%Y%m%d") if start_date else dt2 - datetime.timedelta(days=5)
            dt_range = pd.date_range(start=dt1.strftime("%Y%m%d"), end=dt2.strftime("%Y%m%d"), freq="B")
            days = len(dt_range)
        except Eception as e:
            print(e)
        
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)

        self.start_app_in_background(app)

        ## We'll need to change sec_type and exchange, if we want to check other instruments than stocks.
        contract = self.createContract(symbol=ticker)

        if not end_date:
            end_date = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        if days > 261:
            app.reqHistoricalData(reqId=1,
                                  contract=contract,
                                  endDateTime=dt2.strftime("%Y%m%d %H:%M:%S"),
                                  durationStr="%d Y"%math.ceil(days/261), # [S-Seconds,D-days,W-week,M-Month,Y-Year] default is S.
                                  barSizeSetting=bar_size, # [1 sec,5 secs,15 secs,30 secs,1 min,2 mins,3 mins,5 mins,15 mins,20 mins,30 mins,1 hour,2 hours,3 hours,4 hours,8 hours,1 day, 1 week, 1 month]
                                  whatToShow=measure, # [TRADES,MIDPOINT,BID,ASK,BID_ASK,ADJUSTED_LAST,HISTORICAL_VOLATILITY,OPTION_IMPLIED_VOLATILITY,REBATE_RATE,FEE_RATE,YIELD_BID,YIELD_ASK,YIELD_BID_ASK,YIELD_LAST]
                                  useRTH=0,
                                  formatDate=2,
                                  keepUpToDate=False, chartOptions=[])
            
        else:
            app.reqHistoricalData(reqId=1,
                                  contract=contract,
                                  endDateTime=dt2.strftime("%Y%m%d %H:%M:%S"),
                                  durationStr="%d D"%days, # [S-Seconds,D-days,W-week,M-Month,Y-Year] default is S.
                                  barSizeSetting=bar_size, # [1 sec,5 secs,15 secs,30 secs,1 min,2 mins,3 mins,5 mins,15 mins,20 mins,30 mins,1 hour,2 hours,3 hours,4 hours,8 hours,1 day, 1 week, 1 month]
                                  whatToShow=measure, # [TRADES,MIDPOINT,BID,ASK,BID_ASK,ADJUSTED_LAST,HISTORICAL_VOLATILITY,OPTION_IMPLIED_VOLATILITY,REBATE_RATE,FEE_RATE,YIELD_BID,YIELD_ASK,YIELD_BID_ASK,YIELD_LAST]
                                  useRTH=0,
                                  formatDate=2,
                                  keepUpToDate=False, chartOptions=[])

        time.sleep(sleep_time) #sleep to allow enough time for data to be returned

        df = pd.DataFrame(app.hist_data, columns=['DateTime', 'Open', 'High', 'Low','Close', 'Average', 'Volume', "BarCount"])
        
        ## Datetime format Handling.
        if bar_size in ['1 day', '1 week', '1 month', '1 W', '1 M']:  
            df['DateTime'] = pd.to_datetime(df['DateTime']) 
        else:
            df['DateTime'] = pd.to_datetime(df['DateTime'], unit='s') 
        
        df = df.set_index("DateTime")
        df = df[df.index >= "%d-%d-%d"%(dt1.year, dt1.month, dt1.day)]
        
        app.disconnect()

        return df
    
    def retrieve_ohlc_data_for_tickers(self, tickers = ["AAPL",], start_date="", end_date='', bar_size="1 hour", measure="TRADES", sleep_time=1.5):
        if not start_date:
            print("Start Date Not Provided. Setting Start Date to Default 5 Days back from End Date.")
         
        try:
            dt2 = datetime.datetime.now() if not end_date else datetime.datetime.strptime(end_date, "%Y%m%d")
            dt2 = dt2 + datetime.timedelta(days=1)
            dt1 = datetime.datetime.strptime(start_date, "%Y%m%d") if start_date else dt2 - datetime.timedelta(days=5)
            dt_range = pd.date_range(start=dt1.strftime("%Y%m%d"), end=dt2.strftime("%Y%m%d"), freq="B")
            days = len(dt_range)
        except Eception as e:
            print(e)
        
        if not end_date:
            end_date = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
            
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)

        self.start_app_in_background(app)
        
        tickers_data = {}
        
        for ticker in tickers:
            contract = self.createContract(symbol=ticker)
            if days > 261:
                app.reqHistoricalData(reqId=1,
                                  contract=contract,
                                  endDateTime=dt2.strftime("%Y%m%d %H:%M:%S"),
                                  durationStr="%d Y"%math.ceil(days/261), # [S-Seconds,D-days,W-week,M-Month,Y-Year] default is S.
                                  barSizeSetting=bar_size, # [1 sec,5 secs,15 secs,30 secs,1 min,2 mins,3 mins,5 mins,15 mins,20 mins,30 mins,1 hour,2 hours,3 hours,4 hours,8 hours,1 day, 1 week, 1 month]
                                  whatToShow=measure, # [TRADES,MIDPOINT,BID,ASK,BID_ASK,ADJUSTED_LAST,HISTORICAL_VOLATILITY,OPTION_IMPLIED_VOLATILITY,REBATE_RATE,FEE_RATE,YIELD_BID,YIELD_ASK,YIELD_BID_ASK,YIELD_LAST]
                                  useRTH=0,
                                  formatDate=2,
                                  keepUpToDate=False, chartOptions=[])
            else:
                app.reqHistoricalData(reqId=2,
                                      contract=contract,
                                      endDateTime=dt2.strftime("%Y%m%d %H:%M:%S"),
                                      durationStr="%d D"%days, # [S-Seconds,D-days,W-week,M-Month,Y-Year] default is S.
                                      barSizeSetting=bar_size, # [1 sec,5 secs,15 secs,30 secs,1 min,2 mins,3 mins,5 mins,15 mins,20 mins,30 mins,1 hour,2 hours,3 hours,4 hours,8 hours,1 day, 1 week, 1 month]
                                      whatToShow=measure, # [TRADES,MIDPOINT,BID,ASK,BID_ASK,ADJUSTED_LAST,HISTORICAL_VOLATILITY,OPTION_IMPLIED_VOLATILITY,REBATE_RATE,FEE_RATE,YIELD_BID,YIELD_ASK,YIELD_BID_ASK,YIELD_LAST]
                                      useRTH=0,
                                      formatDate=2,
                                      keepUpToDate=False, chartOptions=[])

            time.sleep(sleep_time) #sleep to allow enough time for data to be returned
            
        
            df = pd.DataFrame(app.hist_data, columns=['DateTime', 'Open', 'High', 'Low','Close', 'Average', 'Volume', "BarCount"])
        
            ## Datetime format Handling.
            if bar_size in ['1 day', '1 week', '1 month', '1 W', '1 M']:  
                df['DateTime'] = pd.to_datetime(df['DateTime']) 
            else:
                df['DateTime'] = pd.to_datetime(df['DateTime'], unit='s') 

            df = df.set_index("DateTime")
            df = df[df.index >= "%d-%d-%d"%(dt1.year, dt1.month, dt1.day)]
            
            tickers_data[ticker] = df.copy()
            
            app.hist_data = []
            
        final_df = None
        
        for ticker, data in tickers_data.items():
            if isinstance(final_df, pd.DataFrame):
                temp_df = data[["Close"]].copy().rename(columns={"Close": ticker})
                final_df = final_df.join(temp_df, how="outer")
            else:
                final_df = data[["Close"]].copy().rename(columns={"Close": ticker})
        
        app.disconnect()

        return final_df
    
    def retrieve_tick_data(self, ticker="AAPL", tick_type="", sleep_time=1):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)

        self.start_app_in_background(app)

        ## We'll need to change sec_type and exchange, if we want to check other instruments than stocks.
        contract = self.createContract(symbol=ticker)
        app.reqMktData(1, contract, tick_type, False, False, [])
        
        time.sleep(sleep_time)
        
        tick_prices = app.tick_prices
        
        app.disconnect()
        
        return tick_prices
        
    def get_last_traded_price(self, ticker="AAPL"):
        prices = self.retrieve_tick_data(ticker=ticker)
        return prices["LAST"]

    def req_account_summary(self):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)
        
        app.reqAccountSummary(reqId=9002,
                              groupName="All",
                              tags="$LEDGER:USD" # [$LEDGER, $LEDGER:USD, $LEDGER:EUR, $LEDGER:ALL]
                             )

        time.sleep(3) #sleep to allow enough time for data to be returned

        df = pd.DataFrame(data=app.acct_summary, columns=["Account", "Tag", "Value", "Currency"])

        app.disconnect()

        return df

    def req_positions(self):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)

        app.reqPositions()

        time.sleep(10)

        cols = ["Account","Symbol","SecType","Currency","Position","Avg cost"]
        df = pd.DataFrame(data=app.positions, columns=cols)

        app.disconnect()

        return df

    def req_pnl(self):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)

        app.reqPnLSingle(17002, "DU2197649", "", 8314)

        time.sleep(10)
        app.disconnect()

    def buy_order(self, ticker, order_type="MKT", qty=1, duration="DAY", lmt_price="", adaptive_algo=False):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)

        while True:
            if isinstance(app.nextorderId, int):
                print('connected')
                break
            else:
                print('waiting for connection')
                time.sleep(1)

        base_order = self.createOrder("BUY", order_type=order_type, qty=qty, duration=duration, lmt_prc=lmt_price)

        if adaptive_algo:
            self.apply_adaptive_algo_params(base_order)

        app.placeOrder(app.nextorderId, self.createContract(ticker), base_order)

        time.sleep(10)

        #if not app.order_status[app.nextorderId] == "Filled":
        #    print('cancelling order')
        #    app.cancelOrder(app.nextorderId)

        #time.sleep(5)
        #print("Order Status : ", app.order_status[app.nextorderId])
        app.disconnect()

    def sell_order(self, ticker, order_type="MKT", qty=1, duration="DAY",  lmt_price="", adaptive_algo=False):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)

        while True:
            if isinstance(app.nextorderId, int):
                print('connected')
                break
            else:
                print('waiting for connection')
                time.sleep(1)

        base_order = self.createOrder("SELL", order_type=order_type, qty=qty, duration=duration, lmt_prc=lmt_price)

        if adaptive_algo:
            self.apply_adaptive_algo_params(base_order)

        app.placeOrder(app.nextorderId, self.createContract(ticker), base_order)

        time.sleep(10)
        #if not app.order_status[app.nextorderId] == "Filled":
        #    print('cancelling order')
        #    app.cancelOrder(app.nextorderId)

        #time.sleep(5)
        #print("Order Status : ", app.order_status[app.nextorderId])
        app.disconnect()

    def retrieve_all_pending_orders(self):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)

        app.reqAllOpenOrders()
        time.sleep(10)

        cols = ["PermId", "ClientId","OrderId","Account","Symbol","SecType","Exchange","Action",
                "OrderType","TotalQty","CashQty","LmtPrice","AuxPrice","Status"]

        df = pd.DataFrame(data=app.pending_orders, columns = cols).drop_duplicates()

        app.disconnect()

        return df

    def cancel_order(self, order_id):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)

        app.cancelOrder(order_id)

        time.sleep(5)

        app.disconnect()    

    def cancel_all_pending_orders(self, ):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)

        app.reqGlobalCancel()

        time.sleep(5)

        app.disconnect()    

    def modify_order(self, order_id, new_duration=None, new_order_type=None, new_qty=None, new_lmt_price=None):
        pending_orders = self.retrieve_all_pending_orders()

        order_to_cancel = pending_orders[pending_orders.OrderId == order_id]

        ticker = order_to_cancel.Symbol.values[0]
        action = order_to_cancel.Action.values[0]
        prev_order_type = order_to_cancel.OrderType.values[0]
        prev_qty = order_to_cancel.TotalQty.values[0]
        prev_lmt_prc = order_to_cancel.LmtPrice.values[0]

        print(ticker, action, prev_order_type, prev_qty, prev_lmt_prc)

        modified_order = self.createOrder(action, 
                                     order_type=new_order_type if new_order_type else prev_order_type,
                                     qty=new_qty if new_qty else prev_qty, 
                                     duration=new_duration,
                                     lmt_prc=new_lmt_price if new_lmt_price else prev_lmt_prc)


        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)

        app.placeOrder(order_id, self.createContract(ticker), modified_order)

        time.sleep(10)
        app.disconnect()

    def modify_portfolio(self, new_portfolio=[], adaptive=False):
        if not new_portfolio:
            print("No New Portfolio Details Provided. Exiting Without Making Any Changes.")
            return

        curr_portfolio = self.req_positions()
        curr_portfolio = list(zip(curr_portfolio["Symbol"], curr_portfolio["Position"]))

        curr_portfolio_dict = dict(curr_portfolio)
        new_portfolio_dict = dict(new_portfolio)

        new_stocks = [stk for stk, qty in new_portfolio]
        curr_stocks = [stk for stk, qty in curr_portfolio]

        ## Present in new and old both
        common_stocks = list(set(new_stocks).intersection(curr_stocks))
        ## Present in only new portfolio
        present_in_new_only = list(set(new_stocks).difference(curr_stocks))
        ## Present in only current portfolio
        present_in_curr_only = list(set(curr_stocks).difference(new_stocks))

        for stock in common_stocks:
            if new_portfolio_dict[stock] > curr_portfolio_dict[stock]:
                diff = new_portfolio_dict[stock] - curr_portfolio_dict[stock]
                ## More stock quantity in new portfolio hence buying difference
                print("Buying Stock : ", stock, " Quantity : ", diff)
                self.buy_order(stock, order_type="MKT", qty=diff, duration="DAY", adaptive_algo=adaptive)
            elif new_portfolio_dict[stock] < curr_portfolio_dict[stock]:
                diff = curr_portfolio_dict[stock] - new_portfolio_dict[stock]
                ## Less stock quantity in new portfolio hence selling difference
                print("Selling Stock : ", stock, " Quantity : ", diff)
                self.sell_order(stock, order_type="MKT", qty=diff, duration="DAY", adaptive_algo=adaptive)
            else:
                continue

        ## Buying stocks only present in new portfolio
        for stock in present_in_new_only:
            print("Buying Stock : ", stock, " Quantity : ", new_portfolio_dict[stock])
            self.buy_order(stock, order_type="MKT", qty=new_portfolio_dict[stock], duration="DAY", adaptive_algo=adaptive)

        ## Selling stocks only present in current portfolio and not new portfolio
        for stock in present_in_curr_only:
            print("Selling Stock : ", stock, " Quantity : ", new_portfolio_dict[stock])
            self.sell_order(stock, order_type="MKT", qty=new_portfolio_dict[stock], duration="DAY", adaptive_algo=adaptive)
            
    def get_available_trading_hours(self, ticker="AAPL"):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)

        self.start_app_in_background(app)
    
        app.reqContractDetails(10004, self.createContract(ticker))
        
        time.sleep(1)

        trading_hours = app.trading_hours
        
        app.disconnect()
        
        return trading_hours
    
    def get_latest_price_for_tickers(self, tickers, sleep_time=0.2):
        app = IBapi()
        app.connect('127.0.0.1', 7497, 123)
        self.start_app_in_background(app)
        latest_price = []
        for ticker in tickers:
            try:
                contract = self.createContract(symbol=ticker)
                app.reqMktData(1, contract, "", False, False, [])
                
                time.sleep(sleep_time)
        
                tick_prices = app.tick_prices
                latest_price.append(price["LAST"])
                app.tick_prices = {}
            except:
                latest_price.append(None)
            
        app.disconnect()
                
        now = datetime.datetime.now()
        return pd.DataFrame([latest_price], columns=histotical_snp_price_df.columns, index=[datetime.datetime(now.year, now.month, now.day)])

In [15]:
trading = Trading()

df = trading.retrieve_ohlc_data(ticker="AAPL", start_date="20200903", end_date="20201005", bar_size="1 day")
df

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  329


ERROR 1 162 Historical Market Data Service error message:No market data permissions for ISLAND STK


,Open,High,Low,Close,Average,Volume,BarCount
DateTime,,,,,,,


In [6]:
trading = Trading()

df = trading.retrieve_ohlc_data(ticker="AMZN", start_date="20100101", end_date="20201008", bar_size="1 day")
df

ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


ERROR 1 162 Historical Market Data Service error message:No market data permissions for ISLAND STK


,Open,High,Low,Close,Average,Volume,BarCount
DateTime,,,,,,,


In [11]:
trading.req_account_summary()

ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


,Account,Tag,Value,Currency
0,DU2197649,Currency,USD,USD
1,DU2197649,CashBalance,1001954.72,USD
2,DU2197649,TotalCashBalance,1001954.72,USD
3,DU2197649,AccruedCash,0.00,USD
4,DU2197649,StockMarketValue,0.00,USD
5,DU2197649,OptionMarketValue,0.00,USD
6,DU2197649,FutureOptionValue,0.00,USD
7,DU2197649,FuturesPNL,0.00,USD
8,DU2197649,NetLiquidationByCurrency,1001954.72,USD
9,DU2197649,UnrealizedPnL,0.00,USD


In [8]:
trading.req_pnl()

ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


In [23]:
trading.req_positions()

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  44


,Account,Symbol,SecType,Currency,Position,Avg cost
0,DU2197649,FB,STK,USD,100.0,256.306403
1,DU2197649,AAPL,STK,USD,313.0,113.934168
2,DU2197649,GOOG,STK,USD,10.0,1467.908230
3,DU2197649,SPY,STK,USD,100.0,333.846703


In [37]:
trading = Trading()
trading.get_last_traded_price("AMZN")

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


3197.0

In [25]:
trading.buy_order("AMZN", order_type="LMT", qty=5, duration="GTC", lmt_price=3110)

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  44
connected
orderStatus - orderid:  44 status:  Submitted filled  0.0 remaining  5.0 lastFillPrice  0.0


In [28]:
trading.retrieve_all_pending_orders()

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  45
OpenOrderEnd


,PermId,ClientId,OrderId,Account,Symbol,SecType,Exchange,Action,OrderType,TotalQty,CashQty,LmtPrice,AuxPrice,Status


In [6]:
trading.cancel_order(11)

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


orderStatus - orderid:  9 status:  PreSubmitted filled  0.0 remaining  1.0 lastFillPrice  0.0
orderStatus - orderid:  11 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  5.0 lastFillPrice  0.0
orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd
The next valid order id is:  14
orderStatus - orderid:  11 status:  PendingCancel filled  0.0 remaining  10.0 lastFillPrice  0.0


ERROR 11 202 Order Canceled - reason:


orderStatus - orderid:  11 status:  Cancelled filled  0.0 remaining  10.0 lastFillPrice  0.0


In [51]:
pending_orders = trading.retrieve_all_pending_orders()
pending_orders

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2103 Market data farm connection is broken:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


orderStatus - orderid:  9 status:  PendingSubmit filled  0.0 remaining  25.0 lastFillPrice  0.0
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  5.0 lastFillPrice  0.0
orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd
The next valid order id is:  14
orderStatus - orderid:  9 status:  PendingSubmit filled  0.0 remaining  25.0 lastFillPrice  0.0
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  5.0 lastFillPrice  0.0
orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd


ERROR -1 2103 Market data farm connection is broken:usfarm.nj


,PermId,ClientId,OrderId,Account,Symbol,SecType,Exchange,Action,OrderType,TotalQty,CashQty,LmtPrice,AuxPrice,Status
0,574207421,123,9,DU2197649,AAPL,STK,SMART,BUY,MKT,25.0,0.0,0.0,0.0,PendingSubmit
1,1444299201,123,13,DU2197649,AMZN,STK,SMART,BUY,LMT,5.0,0.0,3110.0,0.0,PreSubmitted
2,574207422,123,10,DU2197649,GOOG,STK,SMART,BUY,MKT,10.0,0.0,0.0,0.0,PreSubmitted
3,574207424,123,12,DU2197649,SPY,STK,SMART,BUY,LMT,10.0,0.0,330.0,0.0,PreSubmitted


In [52]:
trading.modify_order(13, new_duration="GTC",new_qty=50, new_lmt_price=3111)

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


orderStatus - orderid:  9 status:  PendingSubmit filled  0.0 remaining  25.0 lastFillPrice  0.0
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  5.0 lastFillPrice  0.0
orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd
The next valid order id is:  14
orderStatus - orderid:  9 status:  PendingSubmit filled  0.0 remaining  25.0 lastFillPrice  0.0
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  5.0 lastFillPrice  0.0
orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd


ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


AMZN BUY LMT 5.0 3110.0
orderStatus - orderid:  9 status:  PendingSubmit filled  0.0 remaining  25.0 lastFillPrice  0.0
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  5.0 lastFillPrice  0.0
orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd
The next valid order id is:  14
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  50.0 lastFillPrice  0.0


In [54]:
pending_orders = trading.retrieve_all_pending_orders()
pending_orders

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


orderStatus - orderid:  9 status:  PendingSubmit filled  0.0 remaining  25.0 lastFillPrice  0.0
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  50.0 lastFillPrice  0.0
orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd
The next valid order id is:  14
orderStatus - orderid:  9 status:  PendingSubmit filled  0.0 remaining  25.0 lastFillPrice  0.0
orderStatus - orderid:  13 status:  PreSubmitted filled  0.0 remaining  50.0 lastFillPrice  0.0
orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd


,PermId,ClientId,OrderId,Account,Symbol,SecType,Exchange,Action,OrderType,TotalQty,CashQty,LmtPrice,AuxPrice,Status
0,574207421,123,9,DU2197649,AAPL,STK,SMART,BUY,MKT,25.0,0.0,0.0,0.0,PendingSubmit
1,1444299201,123,13,DU2197649,AMZN,STK,SMART,BUY,LMT,50.0,0.0,3111.0,0.0,PreSubmitted
2,574207422,123,10,DU2197649,GOOG,STK,SMART,BUY,MKT,10.0,0.0,0.0,0.0,PreSubmitted
3,574207424,123,12,DU2197649,SPY,STK,SMART,BUY,LMT,10.0,0.0,330.0,0.0,PreSubmitted


In [14]:
trading.modify_portfolio([('AAPL', 100), ('GOOG', 10), ('AMZN', 2), ('SPY', 10)])

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


orderStatus - orderid:  10 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
orderStatus - orderid:  12 status:  PreSubmitted filled  0.0 remaining  10.0 lastFillPrice  0.0
OpenOrderEnd
The next valid order id is:  14
[('SPY', 100.0)]
{'AAPL': 100, 'GOOG': 10, 'AMZN': 2, 'SPY': 10}
{'SPY': 100.0}
['SPY']
['GOOG', 'AAPL', 'AMZN']
[]
90.0


In [29]:
trading.buy_order("AAPL", "LMT", 100, duration="DAY", lmt_price=114.12)

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  45
connected
orderStatus - orderid:  45 status:  PreSubmitted filled  0.0 remaining  100.0 lastFillPrice  0.0
Order Executed:  -1 AAPL STK USD 0000e0d5.5f967e89.01.01 45 100.0 2
orderStatus - orderid:  45 status:  Filled filled  100.0 remaining  0.0 lastFillPrice  113.94
orderStatus - orderid:  45 status:  Filled filled  100.0 remaining  0.0 lastFillPrice  113.94


In [39]:
trading.get_last_traded_price("AAPL")

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


115.27

In [29]:
trading = Trading()

trading.get_available_trading_hours()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


{'Date': 'October 09 - 2020',
 'Regular Trading Session': '0930 - 1600',
 'Total Available Hours': '0400 - 2000'}

In [177]:
from ibapi.ticktype import TickTypeEnum

for i in range(91):
	print(TickTypeEnum.to_str(i), i)

BID_SIZE 0
BID 1
ASK 2
ASK_SIZE 3
LAST 4
LAST_SIZE 5
HIGH 6
LOW 7
VOLUME 8
CLOSE 9
BID_OPTION_COMPUTATION 10
ASK_OPTION_COMPUTATION 11
LAST_OPTION_COMPUTATION 12
MODEL_OPTION 13
OPEN 14
LOW_13_WEEK 15
HIGH_13_WEEK 16
LOW_26_WEEK 17
HIGH_26_WEEK 18
LOW_52_WEEK 19
HIGH_52_WEEK 20
AVG_VOLUME 21
OPEN_INTEREST 22
OPTION_HISTORICAL_VOL 23
OPTION_IMPLIED_VOL 24
OPTION_BID_EXCH 25
OPTION_ASK_EXCH 26
OPTION_CALL_OPEN_INTEREST 27
OPTION_PUT_OPEN_INTEREST 28
OPTION_CALL_VOLUME 29
OPTION_PUT_VOLUME 30
INDEX_FUTURE_PREMIUM 31
BID_EXCH 32
ASK_EXCH 33
AUCTION_VOLUME 34
AUCTION_PRICE 35
AUCTION_IMBALANCE 36
MARK_PRICE 37
BID_EFP_COMPUTATION 38
ASK_EFP_COMPUTATION 39
LAST_EFP_COMPUTATION 40
OPEN_EFP_COMPUTATION 41
HIGH_EFP_COMPUTATION 42
LOW_EFP_COMPUTATION 43
CLOSE_EFP_COMPUTATION 44
LAST_TIMESTAMP 45
SHORTABLE 46
FUNDAMENTAL_RATIOS 47
RT_VOLUME 48
HALTED 49
BID_YIELD 50
ASK_YIELD 51
LAST_YIELD 52
CUST_OPTION_COMPUTATION 53
TRADE_COUNT 54
TRADE_RATE 55
VOLUME_RATE 56
LAST_RTH_TRADE 57
RT_HISTORICAL_VO

In [4]:
trading = Trading()

#Legal ones for (STK) are: 
# 100(Option Volume)
# 101(Option Open Interest)
# 105(Average Opt Volume)
# 106(impvolat)
# 165(Misc. Stats)
# 221/220(Creditman Mark Price)
# 225(Auction)
# 232/221(Pl Price)
# 233(RTVolume)
# 236(inventory)
# 258/47(Fundamentals)
# 292(Wide_news)
# 293(TradeCount)
# 294(TradeRate)
# 295(VolumeRate)
# 318(LastRTHTrade)
# 375(RTTrdVolume)
# 411(rthistvol)
# 456/59(IBDividends)
# 460(Bond Factor Multiplier)
# 576(EtfNavBidAsk(navbidask))
# 577(EtfNavLast(navlast))
# 578(EtfNavClose(navclose))
# 587(Pl Price Delayed)
# 588(Futures Open Interest)
# 595(Short-Term Volume X Mins)
# 614(EtfNavMisc(high/low)),619(Creditman Slow Mark Price),623(EtfFrozenNavLast(fznavlast))

trading.retrieve_tick_data(ticker="AMZN", tick_type="")

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


{'BID': 3208.0, 'ASK': 3210.72, 'LAST': 3210.0, 'CLOSE': 3190.55}

In [5]:
#trading.retrieve_tick_data(ticker="AMZN", tick_type="165")
trading.retrieve_tick_data(ticker="AMZN", tick_type="225")

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


{'BID': 3208.0,
 'ASK': 3211.0,
 'LAST': 3210.0,
 'CLOSE': 3190.55,
 'AUCTION_PRICE': 3209.5}

In [45]:
trading = Trading()

df = trading.retrieve_ohlc_data_for_tickers(tickers=["AMZN", "AAPL", "GOOG", "SPY", "FB"])
df.head()

ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Start Date Not Provided. Setting Start Date to Default 5 Days back from End Date.
The next valid order id is:  17


,AMZN,AAPL,GOOG,SPY,FB
DateTime,,,,,
2020-10-05 08:00:00,3150.00,114.30,1464.00,335.68,261.83
2020-10-05 09:00:00,3150.00,114.40,1468.44,336.13,262.22
2020-10-05 10:00:00,3165.12,114.53,1468.44,336.30,262.40
2020-10-05 11:00:00,3147.04,113.98,1467.90,336.10,262.00
2020-10-05 12:00:00,3145.00,113.89,1465.02,336.05,261.85


In [53]:
trading = Trading()

df = trading.retrieve_ohlc_data_for_tickers(tickers=["AMZN", "AAPL", "GOOG", "SPY", "FB"], 
                                            start_date="20200901",
                                            end_date="20201007",
                                            bar_size="1 day"
                                           )
df.head()

ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  17


,AMZN,AAPL,GOOG,SPY,FB
DateTime,,,,,
2020-09-01,3513.5,135.00,1667.17,353.57,296.70
2020-09-02,3529.0,131.54,1728.28,358.43,302.85
2020-09-03,3314.0,117.19,1627.47,343.27,285.25
2020-09-04,3250.0,120.00,1580.35,342.00,281.40
2020-09-08,3139.0,112.48,1522.31,332.30,269.26


In [17]:
trading = Trading()

df = trading.retrieve_ohlc_data_for_tickers(tickers=["AMZN", "AAPL", "GOOG", "SPY", "FB"], 
                                            start_date="20201002",
                                            end_date="20201007",
                                            bar_size="1 hour",
                                            sleep_time=2
                                           )
df.head()

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


,AMZN,AAPL,GOOG,SPY,FB
DateTime,,,,,
2020-10-02 08:00:00,3172.00,114.27,NaN,332.85,262.00
2020-10-02 09:00:00,3160.00,113.46,NaN,331.34,260.02
2020-10-02 10:00:00,3155.00,114.00,NaN,332.17,260.95
2020-10-02 11:00:00,3144.00,112.92,1459.88,331.36,260.00
2020-10-02 12:00:00,3150.23,113.28,1462.55,331.50,260.64


In [22]:
trading = Trading()

df = trading.retrieve_ohlc_data_for_tickers(tickers=["AMZN", "AAPL", "GOOG", "SPY", "FB"], 
                                            start_date="20100101",
                                            end_date="20201008",
                                            bar_size="1 day",
                                            sleep_time=6
                                           )
df

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  88


,AMZN,AAPL,GOOG,SPY,FB
DateTime,,,,,
2010-01-04,133.91,7.67,NaN,113.21,NaN
2010-01-05,134.50,7.66,NaN,113.60,NaN
2010-01-06,132.26,7.56,NaN,113.79,NaN
2010-01-07,129.92,7.52,NaN,114.23,NaN
2010-01-08,133.10,7.57,NaN,114.70,NaN
...,...,...,...,...,...
2020-10-02,3114.00,112.58,1453.99,332.64,260.00
2020-10-05,3175.80,115.76,1481.00,338.84,263.40
2020-10-06,3098.00,113.20,1445.65,334.62,256.78


In [14]:
df = pd.read_csv("S&P500-Symbols.csv")
symbols = df.Symbol.tolist()

trading = Trading()

df = trading.retrieve_ohlc_data_for_tickers(tickers=symbols, 
                                            start_date="20201001",
                                            end_date="20201014",
                                            bar_size="1 day",
                                            #sleep_time=1
                                           )
df

ERROR:ibapi.wrapper:ERROR 2 504 Not connected
ERROR:ibapi.wrapper:ERROR 2 504 Not connected
ERROR:ibapi.wrapper:ERROR 2 504 Not connected
ERROR:ibapi.wrapper:ERROR 2 504 Not connected
ERROR:ibapi.wrapper:ERROR 2 504 Not connected


KeyboardInterrupt: 

In [7]:
from yahoo_fin import stock_info as finance
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")

In [8]:
'''
df = pd.read_csv("S&P500-Symbols.csv")
symbols = df.Symbol.tolist()

dfs = []
failed_tickers = []

for ticker in symbols:
    try:
        temp_df = finance.get_data(ticker, "20100101", "20201014")
        temp_df = temp_df.rename(columns = {"close":ticker})
        dfs.append(temp_df[[ticker]])
    except:
        failed_tickers.append(ticker)
        print("Failed Ticker", ticker)
        
final_df = pd.concat(dfs, axis=1)
final_df.to_csv("snp_close_prices.csv")

'''

'\ndf = pd.read_csv("S&P500-Symbols.csv")\nsymbols = df.Symbol.tolist()\n\ndfs = []\nfailed_tickers = []\n\nfor ticker in symbols:\n    try:\n        temp_df = finance.get_data(ticker, "20100101", "20201014")\n        temp_df = temp_df.rename(columns = {"close":ticker})\n        dfs.append(temp_df[[ticker]])\n    except:\n        failed_tickers.append(ticker)\n        print("Failed Ticker", ticker)\n        \nfinal_df = pd.concat(dfs, axis=1)\nfinal_df.to_csv("snp_close_prices.csv")\n\n'

In [10]:
histotical_snp_price_df = pd.read_csv("snp_close_prices.csv", index_col=0, parse_dates=True)
histotical_snp_price_df

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2010-01-04,22.389128,4.77,40.380001,7.643214,NaN,26.629999,8.740000,26.129908,42.070000,37.090000,...,25.379999,69.150002,35.330002,22.740448,NaN,25.226456,60.020000,28.670000,13.330000,NaN
2010-01-05,22.145924,5.31,40.139999,7.656428,NaN,26.440001,8.530000,25.918797,42.330002,37.700001,...,25.059999,69.419998,34.910000,22.766798,NaN,25.140186,61.919998,28.620001,13.800000,NaN
2010-01-06,22.067240,5.09,40.490002,7.534643,NaN,26.190001,8.400000,26.062737,42.779999,37.619999,...,24.889999,70.019997,35.139999,22.555994,NaN,24.960461,61.900002,28.400000,15.000000,NaN
2010-01-07,22.038626,5.24,40.480000,7.520714,NaN,25.770000,8.400000,26.278646,42.740002,36.889999,...,24.639999,69.800003,35.599998,22.661396,NaN,24.953272,63.320000,27.690001,16.680000,NaN
2010-01-08,22.031473,5.14,40.639999,7.570714,NaN,26.049999,8.230000,26.412991,42.570000,36.689999,...,25.000000,69.519997,35.599998,22.582346,NaN,24.960461,61.990002,27.600000,16.410000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-07,103.879997,13.07,156.809998,115.080002,87.070000,95.589996,268.540009,107.739998,223.449997,493.149994,...,105.830002,33.500000,45.049999,20.230000,88.199997,93.879997,144.830002,276.579987,32.150002,159.910004
2020-10-08,104.160004,13.16,157.660004,114.970001,87.339996,98.489998,271.200012,108.540001,225.610001,490.839996,...,105.989998,35.259998,46.060001,20.160000,89.830002,95.410004,146.660004,279.089996,32.139999,163.169998
2020-10-09,105.760002,13.20,155.320007,116.970001,87.699997,96.860001,273.559998,109.650002,228.690002,502.160004,...,120.940002,34.740002,45.500000,19.879999,90.589996,95.489998,147.990005,284.000000,31.580000,165.429993
2020-10-12,105.430000,12.92,156.250000,124.400002,88.320000,96.860001,277.850006,111.050003,230.589996,510.890015,...,118.900002,34.630001,46.000000,20.150000,91.870003,95.980003,149.149994,288.250000,31.830000,168.550003


In [16]:
%%time

def update_snp_hist_data(histotical_snp_price_df, start_dt, end_dt):
    dfs = []
    failed_tickers = []
    for ticker in histotical_snp_price_df.columns:
        try:
            temp_df = finance.get_data(ticker, start_dt, end_dt)
            temp_df = temp_df.rename(columns = {"close":ticker})[[ticker]].drop_duplicates()
            dfs.append(temp_df[[ticker]])
        except:
            failed_tickers.append(ticker)
            print("Failed Ticker", ticker)
    
    curr_df = pd.concat(dfs, axis=1)
    final_df = pd.concat((histotical_snp_price_df[:-2], curr_df))
    final_df.to_csv("snp_close_prices.csv")
    return final_df

start_dt = histotical_snp_price_df.index[-2].strftime("%Y%m%d")
end_dt = datetime.datetime.now().strftime("%Y%m%d")

histotical_snp_price_df = update_snp_hist_data(histotical_snp_price_df, start_dt, end_dt)

20201012 20201014
Failed Ticker ETFC
CPU times: user 28.1 s, sys: 745 ms, total: 28.8 s
Wall time: 4min 26s


In [4]:
histotical_snp_price_df = pd.read_csv("snp_close_prices.csv", index_col=0, parse_dates=True)
histotical_snp_price_df

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2010-01-04,22.389128,4.77,40.380001,7.643214,NaN,26.629999,8.740000,26.129908,42.070000,37.090000,...,25.379999,69.150002,35.330002,22.740448,NaN,25.226456,60.020000,28.670000,13.330000,NaN
2010-01-05,22.145924,5.31,40.139999,7.656428,NaN,26.440001,8.530000,25.918797,42.330002,37.700001,...,25.059999,69.419998,34.910000,22.766798,NaN,25.140186,61.919998,28.620001,13.800000,NaN
2010-01-06,22.067240,5.09,40.490002,7.534643,NaN,26.190001,8.400000,26.062737,42.779999,37.619999,...,24.889999,70.019997,35.139999,22.555994,NaN,24.960461,61.900002,28.400000,15.000000,NaN
2010-01-07,22.038626,5.24,40.480000,7.520714,NaN,25.770000,8.400000,26.278646,42.740002,36.889999,...,24.639999,69.800003,35.599998,22.661396,NaN,24.953272,63.320000,27.690001,16.680000,NaN
2010-01-08,22.031473,5.14,40.639999,7.570714,NaN,26.049999,8.230000,26.412991,42.570000,36.689999,...,25.000000,69.519997,35.599998,22.582346,NaN,24.960461,61.990002,27.600000,16.410000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-07,103.879997,13.07,156.809998,115.080002,87.070000,95.589996,268.540009,107.739998,223.449997,493.149994,...,105.830002,33.500000,45.049999,20.230000,88.199997,93.879997,144.830002,276.579987,32.150002,159.910004
2020-10-08,104.160004,13.16,157.660004,114.970001,87.339996,98.489998,271.200012,108.540001,225.610001,490.839996,...,105.989998,35.259998,46.060001,20.160000,89.830002,95.410004,146.660004,279.089996,32.139999,163.169998
2020-10-09,105.760002,13.20,155.320007,116.970001,87.699997,96.860001,273.559998,109.650002,228.690002,502.160004,...,120.940002,34.740002,45.500000,19.879999,90.589996,95.489998,147.990005,284.000000,31.580000,165.429993
2020-10-12,105.430000,12.92,156.250000,124.400002,88.320000,96.860001,277.850006,111.050003,230.589996,510.890015,...,118.900002,34.630001,46.000000,20.150000,91.870003,95.980003,149.149994,288.250000,31.830000,168.550003


In [33]:
%%time

trading = Trading()

curr_trade_prices = trading.get_latest_price_for_tickers(histotical_snp_price_df.columns)
final_df = pd.concat((histotical_snp_price_df, curr_trade_prices))
final_df

In [57]:
def add_todays_price_data_to_historical_data():
    from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

    ppe = ThreadPoolExecutor(max_workers=16)

    results = []
    for ticker in histotical_snp_price_df.columns.tolist():
        results.append(ppe.submit(finance.get_live_price, ticker))


    latest_price = [result.result() for result in results]
    now = datetime.datetime.now()

    curr_trade_prices = pd.DataFrame([latest_price], columns=histotical_snp_price_df.columns, index=[datetime.datetime(now.year, now.month, now.day)])
    final_df = pd.concat((histotical_snp_price_df, curr_trade_prices))
    return final_df

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 19.1 µs


In [58]:
%%time

add_todays_price_data_to_historical_data()

KeyboardInterrupt: 